# Détection automatique d'erreurs d'OCR

## Imports

In [1]:
import nltk
from thefuzz import fuzz
from thefuzz import process
from Levenshtein import distance

Nous allons d'abord utiliser la distance de Levenshtein, selon sa formule originale (1965)

In [2]:
distance("bulletin", "ballotins")

3

Nous allons ensuite faire appel à la librairie TheFuzz (anciennement fuzzywuzzy) qui repose sur python-Levenshtein (https://github.com/seatgeek/thefuzz).
Celle-ci permet de normaliser le résultat en calculant un ratio (pourcentage) de similarité entre deux chaînes de caractères :

In [5]:
print(fuzz.ratio("bulletin", "ballotins"))
print(fuzz.ratio("le", "la"))

71
50


La fonction  `partial_ratio` se focalise sur la sous-chaîne (substring) la plus similaire : 

In [6]:
fuzz.partial_ratio("Grand", "Grand-Place")
#output = 100 car Grand est 100% commun avec Grand-Place

100

La fonction `token_sort_ratio` ignore l'ordre des mots :

In [7]:
fuzz.token_sort_ratio("archives de la ville de Bruxelles", "de Bruxelles la ville de archives")

100

La fonction `token_set_ratio` ignore les répétitions de mots :

In [8]:
fuzz.token_set_ratio("plus jamais ça !", "plus jamais Jamais JAMAIS ça !!!")

100

Chargeons à présent le fichier `all.txt` (tronqué) : 

In [12]:
data_path = "../data/all.txt"
LIMIT = 50000000
text = open(data_path, "r", encoding="utf-8").read()[:LIMIT]

tokens = nltk.wordpunct_tokenize(text)
print(f"{len(tokens)} words (tokens) found")

vocab = set(tokens)
print(f"{len(vocab)} different word forms")

12662126 words (tokens) found
790907 different word forms


Nous pouvons identifier les variantes d'un mot par ordre décroissant de similarité : 

In [15]:
word = "bruxelles"
process.extract(word, vocab, limit=15)

[('bruxelles', 100),
 ('BrUxelles', 100),
 ('BRUXELLES', 100),
 ('BruxelLes', 100),
 ('àBruxelles', 100),
 ('BRUXElLES', 100),
 ('èëBruxelles', 100),
 ('BruxelleS', 100),
 ('Bruxellesà', 100),
 ('üBruxelles', 100),
 ('Bruxellesâ', 100),
 ('Bruxelles', 100),
 ('BRUXELLESi', 95),
 ('iBruxelles', 95),
 ('Bruxelles3', 95)]

Les caractères accentués ne sont pas pris en compte pour le calcul du score, mais cela reste une bonne base pour récupérer les occurrences mal OCRisées d'un mot.